In [1]:
import os
import pandas as pd
import glob
from pathlib import PurePosixPath
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data_files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(os.getcwd() + '\\data') for f in filenames]

In [3]:
data_titles = [text[text.index('data')+5:].replace('\\', '/') for text in data_files]

In [4]:
tfidf_vectorizer = TfidfVectorizer(input='filename', decode_error='ignore')
tfidf_vector = tfidf_vectorizer.fit_transform(data_files)

In [5]:
tfidf_vector = tfidf_vector.astype('float32')

In [6]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=data_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [7]:
tfidf_df.T['GP/part6/msg9150.eml'].sort_values(ascending=False).head(25)

br             0.450207
nbsp           0.256397
idris          0.247658
abass          0.247658
my             0.245418
yakubu         0.178371
the            0.174478
handicapped    0.140560
of             0.124310
beign          0.123829
eessth         0.123829
less           0.118239
wealth         0.116935
privileged     0.107877
and            0.106329
vanity         0.103164
god            0.100608
am             0.092338
to             0.091515
quot           0.087965
in             0.087946
live           0.084252
personal       0.083253
for            0.082881
joy            0.081082
Name: GP/part6/msg9150.eml, dtype: float32

In [8]:
tfidf_df.T['lokay-m/articles/1'].sort_values(ascending=False).head(25)

enron          0.531350
skygen         0.334562
calpine        0.305998
com            0.276489
panda          0.161816
mw             0.142102
the            0.116729
michael        0.115139
of             0.103342
alliance       0.101510
oneta          0.100369
energy         0.094653
development    0.084664
turbines       0.084249
polsky         0.083619
steve          0.079132
projects       0.071924
under          0.069876
will           0.069754
and            0.067032
larry          0.063496
john           0.063084
ken            0.061154
frank          0.059636
gas            0.058011
Name: lokay-m/articles/1, dtype: float32

In [9]:
import re
from sklearn.ensemble import RandomForestClassifier

In [10]:
spam_labels = [1 if re.match('GP\/part[0-9]*\/.*\.eml', filename) else 0 for filename in data_titles]

In [11]:
X_train = tfidf_df.filter(regex='GP\/part[0-9]\/.*\.eml', axis=0)

In [ ]:
X

In [12]:
# X_train = df.filter(regex='string', axis=0)

In [13]:
rfc_model = RandomForestClassifier(n_estimators=100)